It is highly recommended to use a powerful **GPU**, you can use it for free uploading this notebook to [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb).
<table align="center">
 <td align="center"><a target="_blank" href="https://colab.research.google.com/github/ezponda/intro_deep_learning/blob/main/class/CNN/Visualizing_What_CNNs_Learn.ipynb">
        <img src="https://colab.research.google.com/img/colab_favicon_256px.png"  width="50" height="50" style="padding-bottom:5px;" />Run in Google Colab</a></td>
  <td align="center"><a target="_blank" href="https://github.com/ezponda/intro_deep_learning/blob/main/class/CNN/Visualizing_What_CNNs_Learn.ipynb">
        <img src="https://github.githubassets.com/images/modules/logos_page/GitHub-Mark.png"  width="50" height="50" style="padding-bottom:5px;" />View Source on GitHub</a></td>
</table>


**Table of Contents**

1. [Introduction](#introduction)
2. [Accessing Layers and Weights in TensorFlow](#accessing_layers)
3. [Visualizing Layer Outputs](#visualizing_outputs)
4. [Methods for Interpretation](#interpretation)
    * 4.1 [Gradient-weighted Class Activation Mapping (Grad-CAM)](#gradcam)
    * 4.2 [Saliency Maps](#saliency_maps)
5. [Conclusion](#conclusion)

<a id='introduction'></a>
# Introduction

Convolutional Neural Networks (CNNs) have been successful in solving complex machine learning problems, particularly in image recognition tasks. Although they are highly effective, they are also often criticized as being black boxes since the learned representations are hard to interpret. Fortunately, we have several visualization techniques to shed some light on what's happening inside these networks.

<a id='accessing_layers'></a>
# Accessing Layers and Weights in TensorFlow


To visualize what CNNs learn, we first need to access the internal components of our CNN model, including its layers and weights. In TensorFlow, the layers of a model can be accessed using the .layers attribute and weights can be accessed using the .get_weights() method.

Here is a simple example of how you can access these components:

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

# Load the VGG16 model
model_VGG16 = VGG16()

model_VGG16.summary()

In [ ]:
# Accessing layers
for layer in model_VGG16.layers:
    print("Layer Name : ", layer.name)

In [ ]:
# Accessing weights of a specific layer
W, b = model_VGG16.layers[1].get_weights()
W.shape, b.shape

<a id='visualizing_outputs'></a>
## 3. Visualizing Layer Outputs



In [ ]:
def read_image(image_path, target_size=None, grayscale=False):
    image = tf.keras.preprocessing.image.load_img(image_path,
                                                  target_size=target_size,
                                                  grayscale=grayscale)
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = image.astype(np.uint8)
    return image


url = 'https://i.ibb.co/q5TFmqh/bird.jpg'
image_path = tf.keras.utils.get_file("bird.jpg", url)
image = read_image(image_path, target_size=(224, 224))
img = np.expand_dims(image, axis=0)
plt.imshow(image, cmap='viridis')
plt.show()

In [ ]:
from tensorflow.keras.models import Model

# Get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = {layer.name: layer.output for layer in model_VGG16.layers}

# Set up a model that returns the activation values for every layer in
# VGG16 (as a dict).
feature_extractor = Model(inputs=model_VGG16.inputs, outputs=outputs_dict)

In [ ]:
## Get the features of the image
features = feature_extractor(img)
print(features.keys())

In [ ]:
# get feature map for first hidden layer
feature_maps = features['block1_conv1']
print('feature_maps first hidden layer shape: ', feature_maps.shape)
# plot all 64 maps in an 8x8 squares
square = 8
ix = 1
plt.figure(figsize=(30, 30))
for _ in range(square):
    for _ in range(square):
        # specify subplot and turn of axis
        ax = plt.subplot(square, square, ix)
        ax.set_xticks([])
        ax.set_yticks([])
        # plot filter channel in viridis or gray
        plt.imshow(feature_maps[0, :, :, ix - 1], cmap='viridis')
        ix += 1
# show the figure
plt.show()

In [ ]:
feature_maps = features['block1_conv2'] 
print('feature_maps shape: ', feature_maps.shape)
for i in range(feature_maps.shape[-1]):
    plt.xticks([])
    plt.yticks([])
    plt.imshow(feature_maps[0, :, :, i], cmap='viridis')
    plt.show()

In [ ]:
feature_maps = features['block1_pool'] 
print('feature_maps shape: ', feature_maps.shape)
for i in range(feature_maps.shape[-1]):
    plt.xticks([])
    plt.yticks([])
    plt.imshow(feature_maps[0, :, :, i], cmap='viridis')
    plt.show()

<a id='interpretation'></a>
# Methods for Interpretation

There are several methods for interpreting the results from CNNs. We will look into two popular ones: Grad-CAM and Saliency Maps.

<a id='imagenet'></a>
## imagenet 

ImageNet is a large dataset of over 14 million labeled images spanning 20,000+ categories. The images were collected from the internet and labeled by human annotators using Amazon's Mechanical Turk crowdsourcing tool.

For the purpose of this tutorial, the ImageNet dataset is particularly relevant because the model we're using (VGG16) was trained on ImageNet. The dataset consists of 1000 different classes, and thus, the model is capable of recognizing 1000 different types of objects.

The ImageNet class index file we download is a dictionary that maps the class indices, which are the output of the model's prediction, to human-readable labels. When we visualize the Grad-CAM heatmaps, we use these labels to understand what the model is recognizing. This is why we need the ImageNet class index in this tutorial.


In [ ]:
import json
import requests

# URL of the ImageNet class index
url = 'https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json'

# Send a HTTP request to the URL
response = requests.get(url)
print(response.status_code)
# If the request is successful, the status code will be 200
if response.status_code == 200:
    # Get the content of the response
    content = response.content

    # Save the json content into a dictionary
    CLASS_INDEX = json.loads(content)

    # Save the dictionary into a local file so you don't have to download it every time
    with open('imagenet_class_index.json', 'w') as f:
        json.dump(CLASS_INDEX, f)

    # Let's test it:
    print(CLASS_INDEX[str(282)])  # it should print ['n02123045', 'tabby'] (a kind of cat)

else:
    print("Failed to download the file. Status code:", response.status_code)


In [ ]:
# Transform CLASS_INDEX to class_name:index
class2index = {value[1]: int(key) for key, value in CLASS_INDEX.items()}

# Let's test it:
class2index


<a id='gradcam'></a>
## Gradient-weighted Class Activation Mapping (Grad-CAM)

Grad-CAM uses the gradients of any target concept flowing into the final convolutional layer to produce a coarse localization map of the important regions in the image. It combines the strengths of the gradient-based localization and class activation mapping to achieve

 better performance.

The computation of Grad-CAM can be outlined as follows:

1. Given an image, forward propagate it through the model to obtain the raw class scores before softmax.
2. Compute the gradients of the class score with respect to feature maps of the final convolutional layer.
3. Global average pool the gradients over the width and height dimensions to obtain the neuron importance weights.
4. The weighted combination of forward activation maps is then followed by a ReLU.

Mathematically, Grad-CAM can be expressed as:

Let $A^k$ be the activation maps and $y^c$ be the class score. The importance weights $\alpha^c_k$ are calculated as:

$$\alpha^c_k = \frac{1}{Z} \sum_i \sum_j \frac{\partial y^c}{\partial A^k_{ij}}$$

where $Z$ is the total number of pixels in each feature map. The localization map $L^c_{Grad-CAM}$ is then obtained as:

$$L^c_{Grad-CAM} = ReLU(\sum_k \alpha^c_k A^k)$$

Example with tf-keras-vis:

In [ ]:
#!pip install tf-keras-vis

In [ ]:
# great_white_shark image
url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/5/56/White_shark.jpg/480px-White_shark.jpg'
image_path = tf.keras.utils.get_file("great_white_shark2.jpg", url)
image = read_image(image_path, target_size=(224, 224))
img = np.expand_dims(image, axis=0)
plt.imshow(image, cmap='viridis')
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
X = preprocess_input(img)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import cm
from tf_keras_vis.gradcam_plus_plus import GradcamPlusPlus
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear
from tf_keras_vis.utils.scores import CategoricalScore

categorical_index = class2index['great_white_shark']

# Create GradCAM++ object
gradcam = GradcamPlusPlus(model_VGG16,
                          model_modifier=ReplaceToLinear(),
                          clone=True)

# Generate cam with GradCAM++
cam = gradcam(CategoricalScore(categorical_index),X,
              penultimate_layer=-1)


plt.imshow(image)
heatmap = np.uint8(cm.jet(cam[0])[..., :3] * 255)
plt.imshow(heatmap, cmap='jet', alpha=0.5) # overlay
plt.axis('off')